In [1]:
import pandas as pd
%store -r melb_df
display(melb_df.head())

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Landsize,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,MonthSale,WeekdaySale
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,...,202.0,126.0,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984",12,5
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,...,156.0,79.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934",2,3
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,...,134.0,150.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944",3,5
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,2017-03-04,2.5,...,94.0,126.0,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969",3,5
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,...,120.0,142.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941",6,5


In [11]:
# Библиотека Pandas предоставляет большое количество возможностей
# для преобразований данных, однако иногда необходимо совершать 
# более сложные манипуляции над столбцами. Например, из столбцов,
# содержащих в себе некоторый текст, 
# необходимо специальным образом извлечь определённые слова,
# даты или числа.

# Для таких случаев Pandas не имеет специальных методов,
# однако позволяет расширить свою функциональность за счёт
# использования пользовательских функций. 

# Мы можем написать некоторую функцию, которая принимает
# на вход один элемент столбца, каким-то образом его обрабатывает
# и возвращает результат, после чего применить эту функцию
# к каждому элементу в столбце 
# с помощью специального метода
# apply().
# В результате применения этой функции будет возвращён объект Series,
# элементы которого будут представлять результат работы этой функции.

# Рассмотрим пример. В наших данных есть столбец с адресами объектов 
# недвижимости. Проблема этого столбца в том, что в нём слишком большое
# количество уникальных значений: почти на каждый объект недвижимости в
# таблице приходится свой уникальный адрес. 
# Убедимся в этом, вычислив количество уникальных значений в столбце 
# с помощью метода 
# nunique():

print(melb_df['Address'].nunique())

13378


In [12]:
# Если мы прогнозируем цену объекта,
# то такое большое количество возможных категорий может плохо сказаться
# на модели, которую мы бы хотели в дальнейшем построить на наших данных.
# Говорят, что такой признак, скорее всего,
# не имеет статистической значимости, 
# потому что не позволяет разделить данные на группы, 
# которые можно сравнить по целевому признаку.

# Из-за таких признаков зависимость между целевым признаком,
# который мы хотим предсказать, и признаками, 
# на основе которых мы делаем предсказание, становится очень сложной.
# При этом точность моделирования при учёте такого признака
# может не повыситься, а даже снизиться,
# а производительность однозначно резко упадёт.

# Обычно подобные признаки удаляют, однако можно поступить умнее:
# давайте извлечём из признака адреса характеристику
# подтипа улицы (улица, шоссе, авеню, бульвар).
# Для этого сначала внимательнее посмотрим на структуру адреса,
# выберем несколько строк столбца Address:

print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [2]:
# Итак, адрес строится следующим образом:
# сначала указывается номер дома и корпус,
# после указывается название улицы,
# а в конце — подтип улицы,
# но в некоторых случаях к подтипу добавляется географическая отметка
# (N — север, S — юг и т. д.), она нам не нужна.
# Для того чтобы выделить подтип улицы, на которой находится объект,
# можно использовать следующую функцию:

# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

# Теперь применим эту функцию к столбцу c адресом.
# Для этого передадим функцию get_street_type 
# в аргумент 
# метода столбца
# apply(). 
# В результате получим объект Series, 
# который положим в переменную street_types:

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

# Обратите внимание, что функция пишется для одного элемента столбца,
# а метод apply()
# применяется к каждому его элементу.
# Используемая функция обязательно должна иметь возвращаемое значение.

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [14]:
# Итак, мы смогли выделить подтип улицы.
# Посмотрим, сколько уникальных значений у нас получилось:

print(street_types.nunique())

56


In [15]:
# У нас есть 56 уникальных значений.
# Однако наш результат можно улучшить.
# Давайте для начала посмотрим на частоту 
# каждого подтипа улицы с помощью метода value_counts:

display(street_types.value_counts())

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Co

In [16]:
# Из данного вывода можно увидеть,
# что есть группа наиболее популярных подтипов улиц,
# а дальше частота подтипов быстро падает.

# В таком случае давайте применим очень распространённый метод
# уменьшения количества уникальных категорий — выделим n подтипов,
# которые встречаются чаще всего,
# а остальные обозначим как 'other' (другие).

# Для этого к результату метода value_counts применим 
# метод 
# nlargest(),
# который возвращает n наибольших значений из Series.
# Зададим n=10,
# т. е. мы хотим отобрать десять наиболее популярных подтипов.
# Извлечём их названия с помощью атрибута index,
# а результат занесём в переменную popular_stypes:

popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


In [17]:
# Попробуйте применить использованные методы последовательно
# и выводить результат, чтобы проследить за логикой преобразования.

# Теперь, когда у нас есть список наиболее популярных подтипов улиц,
# введём lambda-функцию, которая будет проверять,
# есть ли строка x в этом перечне, и, если это так,
# lambda-функция будет возвращать x,
# в противном случае она будет возвращать строку 'other'.
# Наконец, применим такую функцию к Series street_types,
# полученной ранее, а результат определим в новый столбец
# таблицы StreetType:

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [18]:
# Посмотрим на результирующее число уникальных подтипов:

print(melb_df['StreetType'].nunique())

11


In [19]:
# Теперь, у нас нет потребности хранить признак Address,
# так как, если конкретное местоположение объекта всё же
# и влияет на его стоимость,
# то оно определяется столбцами Longitude и Lattitude.
# Удалим его из нашей таблицы:

melb_df = melb_df.drop('Address', axis=1)

# Таким образом, с помощью написания собственных функций
# и их комбинирования с методом apply() из библиотеки Pandas
# мы смогли извлечь информацию из признака с адресом
# и заменить на признак подтипа улицы.

In [5]:
# Мы предлагаем вам самостоятельно разобраться с этой ошибкой:
# попробуйте написать функцию-преобразование
# (lambda-функцию-преобразование),
# которая возвращала бы вместо значений 
# Avenue,
# Boulevard 
# и Parade их топографические сокращения,
# и примените её к данным о подтипах улиц.
Av_list = ['Av','Avenue']
Bvd_list = ['Bvd','Boulevard']
Pde_list = ['Pde','Parade']
melb_df['StreetType'] = melb_df['StreetType'].apply(lambda x: 'Av' if x in Av_list  else x)
melb_df['StreetType'] = melb_df['StreetType'].apply(lambda x: 'Bvd' if x in Bvd_list  else x)
melb_df['StreetType'] = melb_df['StreetType'].apply(lambda x: 'Pde' if x in Pde_list else x)
print(melb_df['StreetType'].nunique())
display(melb_df['StreetType'].value_counts())


53


StreetType
St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: count, dtype: int64

In [6]:
# Ранее, в задании 3.3, мы создали признак WeekdaySale
# в таблице melb_df — день недели продажи.
# Из полученных в задании результатов можно сделать вывод,
# что объекты недвижимости в Мельбурне 
# продаются преимущественно по выходным (суббота и воскресенье).
# Напишите функцию get_weekend(weekday),
# которая принимает на вход элемент столбца WeekdaySale и
# возвращает 1, если день является выходным,
# и 0 — в противном случае,
# и создайте столбец Weekend в таблице melb_df с помощью неё.

# Примените эту функцию к столбцу
# и вычислите среднюю цену объекта недвижимости,
# проданного в выходные дни. Результат округлите до целых.

def get_weekend(weekday):
    res = int(weekday > 4)
    return res 
get_weekend(4)

0

In [7]:
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
x = melb_df[melb_df['Weekend'] == 1]['Price'].mean()
display(round(x))

1081199

In [17]:
# Задание 4.3

# Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df
# следующим образом:
# оставьте в столбце только 49 самых популярных компаний,
# а остальные обозначьте как 'other'.

# Найдите, во сколько раз минимальная цена объектов недвижимости,
# проданных компанией 'Nelson',
# больше минимальной цены объектов, 
# проданных компаниями, обозначенными как 'other'.
# 
# Ответ округлите до десятых.
popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
nelson_min = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
other_min = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
display(round(nelson_min / other_min, 1))

1.3

In [38]:
# Представьте, что вы занимаетесь подготовкой данных о вакансиях
# с платформы hh.ru. В вашем распоряжении имеется таблица,
# в которой с помощью парсинга собраны резюме кандидатов.
# В этой таблице есть текстовый столбец «Опыт работы».
# Пример такого столбца представлен ниже в виде объекта Series.
# Структура текста в столбце фиксирована и не может измениться.

# Напишите функцию get_experience(arg),
# аргументом которой является строка столбца с опытом работы.
# Функция должна возвращать опыт работы в месяцах.
# Не забудьте привести результат к целому числу.

# Обратите внимание, что 
# опыт работы может выражаться
# только в годах 
# или только в месяцах.
# Учтите это при построении своей функции.
# Кроме того, 
# учтите возможные вариации слов месяц (месяца, месяцев)
# и год (года, лет).

# Примените вашу функцию к Series experience_col с помощью метода apply().

def get_experience(arg):
    months_list = ['месяц', 'месяца', 'месяцев'] # список возможных указаний лет опыта
    years_list = ['год', 'года','лет']          # список возможных указаний месяцев опыта
    
    experience_list = arg.split(' ')            # разбиваем строку по пробелу
    
    if len(experience_list) > 4:                # значит опыт указан в годах и месяцах
        experience_in_months = int(experience_list[-4])*12 + int(experience_list[-2])
    else:
        if experience_list[-1] in years_list:                   # если указан опыт только в годах
            experience_in_months = int(experience_list[-2])*12  # перемножаем на 12
        if experience_list[-1] in months_list:                  # если указан опыт только в месяцах                           
            experience_in_months = int(experience_list[-2])    # опыт равен кол-ву месяцев
    return experience_in_months


test_series_1 = pd.Series([
            'Опыт работы 8 лет 3 месяца',
            'Опыт работы 3 года 5 месяцев',
            'Опыт работы 1 год 9 месяцев',
            'Опыт работы 3 месяца',
            'Опыт работы 6 лет'
        ])

test_series_2 = pd.Series([
            'Опыт работы 5 лет',
            'Опыт работы 5 месяцев',
            'Опыт работы 1 год 1 месяц',
            'Опыт работы 3 месяца',
            'Опыт работы 7 лет'
        ])


print(test_series_1.apply(get_experience))
print(test_series_2.apply(get_experience))

0    99
1    41
2    21
3     3
4    72
dtype: int64
0    60
1     5
2    13
3     3
4    84
dtype: int64


In [8]:
%store melb_df

Stored 'melb_df' (DataFrame)
